Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [6]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    !pip install category_encoders==2.*

In [52]:
import numpy as np
import pandas as pd
import category_encoders as ce
from scipy.stats import randint, uniform
from sklearn.feature_selection import f_classif, SelectKBest, VarianceThreshold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [53]:
def dadaWrangle( x):
    x = x.copy()

    x['latitude'] = x['latitude'].replace( -2e-08, 0)
    zerodCols = ['latitude', 'longitude', 
                 'construction_year', 
                 'gps_height','population']
    for cols in zerodCols:
        x[cols] = x[cols].replace( 0, np.nan)
        x[cols + '_MISSING'] = x[cols].isnull()
    
    x['date_recorded'] = pd.to_datetime( x['date_recorded'], 
                                        infer_datetime_format= True)
    x['year_recorded'] = x['date_recorded'].dt.year
    x['month_recorded'] = x['date_recorded'].dt.month
    x['day_recorded'] = x['date_recorded'].dt.day

    ## moved date stratification up so df.drop only needs to be called once

    x = x.drop( columns= ['quantity_group', 'payment_type',     ## duplicates
                          'recorded_by', 'id',                  ## unusable
                          'date_recorded'])                     ## superfluous

    x['age'] = ( x['year_recorded'] - x['construction_year'] )
    x['years_MISSING'] = x['age'].isnull()

    return x

dadaWrangle( train)
dadaWrangle( test)

,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,...,latitude_MISSING,longitude_MISSING,construction_year_MISSING,gps_height_MISSING,population_MISSING,year_recorded,month_recorded,day_recorded,age,years_MISSING
0,0.0,Dmdd,1996.0,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,...,False,False,False,False,False,2013,2,4,1.0,False
1,0.0,Government Of Tanzania,1569.0,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,...,False,False,False,False,False,2013,2,4,13.0,False
2,0.0,NaN,1567.0,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,...,False,False,False,False,False,2013,2,1,3.0,False
3,0.0,Finn Water,267.0,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,...,False,False,False,False,False,2013,1,22,26.0,False
4,500.0,Bruder,1260.0,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,...,False,False,False,False,False,2013,3,27,13.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14353,0.0,Danida,34.0,Da,38.852669,-6.582841,Kwambwezi,0,Wami / Ruvu,Yombo,...,False,False,False,False,False,2011,2,24,23.0,False
14354,1000.0,Hiap,NaN,HIAP,37.451633,-5.350428,Bonde La Mkondoa,0,Pangani,Mkondoa,...,False,False,False,True,False,2011,3,21,17.0,False
14355,0.0,NaN,1476.0,NaN,34.739804,-4.585587,Bwawani,0,Internal,Juhudi,...,False,False,False,False,False,2013,3,4,3.0,False
14356,0.0,Germany,998.0,DWE,35.432732,-10.584159,Kwa John,0,Lake Nyasa,Namakinga B,...,False,False,False,False,False,2013,2,18,4.0,False


In [54]:
target = 'status_group'

## fallInLine.exe commencing...

tFeatures = train.drop( columns= [target, 'id'])
numFeatures = tFeatures.select_dtypes( include= 'number').columns.tolist()
cardiB = tFeatures.select_dtypes( exclude= 'number').nunique()
catFeatures = cardiB[ cardiB <= 50].index.tolist()

features = numFeatures + catFeatures

In [55]:
xTrain = train[ features]
yTrain = train[ target]

xTest = test[ features]

In [57]:
%%time
popeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer( strategy= 'mean'), 
    SelectKBest( f_classif, k= 31), 
    RandomForestClassifier( n_estimators= 100, n_jobs= -1, random_state= 16)
)

k = 10

with np.errstate(divide='ignore',invalid='ignore'):         ## ignore divide by 0 errors
    scored = np.mean( cross_val_score( popeline, xTrain, yTrain, cv= k)) ## scoring= throws error re: yTrain being string vals
#                              scoring= 'neg_mean_absolute_error')
    print( 'MAE for {} folds: {:.5f}'.format( k, scored) )

C:\Users\reall\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [12] are constant.
  UserWarning)
C:\Users\reall\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [12] are constant.
  UserWarning)
C:\Users\reall\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [12] are constant.
  UserWarning)
C:\Users\reall\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [12] are constant.
  UserWarning)
C:\Users\reall\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [12] are constant.
  UserWarning)
C:\Users\reall\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [12] are constant.
  UserWarning)
C:\Users\reall\Anaconda3\lib\site-packages\sklearn\feature_selection\univari

MAE for 10 folds: 0.80678
Wall time: 56.7 s


In [48]:
scored.mean()

0.8067840500004652

In [51]:
print( 'Model Hyperparameters:')
print( popeline.named_steps['randomforestclassifier'])

Model Hyperparameters:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=16, verbose=0,
                       warm_start=False)


In [66]:
from sklearn.model_selection import RandomizedSearchCV

# %%time                    ## "UsageError: Line magic function `%%time` not found." ???
popeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier( random_state= 16)
)

parameterDist = {
    'ordinalencoder__drop_invariant': ['True'],
    'ordinalencoder__handle_missing': ['return_nan'],
    'simpleimputer__strategy': ['mean', 'median'],
    'randomforestclassifier__criterion': ['entropy'],
    'randomforestclassifier__n_estimators': randint( 50, 500),
    'randomforestclassifier__max_features': uniform( 0, 1),
    ## From documentation: Note: the search for a split does not stop 
    ## until at least one valid partition of the node samples is found, 
    ## even if it requires to effectively inspect more than max_features features.
    ## So setting max_features is kiiiinnnnd of pointless...
    
}

searching = RandomizedSearchCV(
    popeline,
    param_distributions = parameterDist,
    n_iter = 10,
    cv = 10,
    scoring = 'neg_mean_absolute_error',
    verbose = 10,
    return_train_score = True,
    n_jobs = -1
)

searching.fit( xTrain, yTrain)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  2.4min


ValueError: could not convert string to float: 'functional'

In [ ]:
print( 'Best hyperparameters:', searching.best_params_)
print( 'Cross validation MAE:', -searching.best_score_)